In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient


import numpy as np
import pandas as pd
import scipy.stats as stats

import datetime

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [2]:
client = MongoClient()
db_eur = client.eur_scraper

collection_eur = db_eur.rates

In [3]:
df_eur = pd.DataFrame(list(collection_eur.find()))

In [4]:
df_eur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   _id        53 non-null     object
 1   year       51 non-null     object
 2   day_month  50 non-null     object
 3   eur_rate   49 non-null     object
dtypes: object(4)
memory usage: 1.8+ KB


In [5]:
# remove Mongo _id column

df_eur.drop(columns = ['_id'], inplace = True)

In [6]:
# drop rows with NaN values

df_eur.dropna(inplace = True)

In [7]:
df_eur.head()

,year,day_month,eur_rate
3,2019,18 Sep.,−0.50
4,2016,16 Mar.,−0.40
5,2015,9 Dec.,−0.30
6,2014,10 Sep.,−0.20
7,2014,11 Jun.,−0.10


In [8]:
df_eur['date'] = df_eur['day_month'] + ' ' + df_eur['year']

In [9]:
def date_converter(date):
    
    """
    PARAMETERS
    date - string in format '2-digit-day month_name 4-digit-year'
    
    RETURNS
    date in new string format ' 4 digit year-2 digit month-2 digit day'
    """
    
    dct_month = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }
    
    lst_date = date.split()
        
    str_yr = lst_date[-1]
    
    str_mo = dct_month[lst_date[1][:3]]
    
    str_d = lst_date[0]
    
    #add zeros before single digit dates
    if len(str_d) ==1:
        str_d += '0'
        str_d = str_d[::-1]
        
    
    str_date = f'{str_yr}-{str_mo}-{str_d}'

    return str_date

In [10]:
df_eur['date'] = [date_converter(d) for d in df_eur['date']]

In [11]:
df_eur['date'] = pd.to_datetime(df_eur['date'], yearfirst = True)

In [12]:
df_eur.head()

,year,day_month,eur_rate,date
3,2019,18 Sep.,−0.50,2019-09-18
4,2016,16 Mar.,−0.40,2016-03-16
5,2015,9 Dec.,−0.30,2015-12-09
6,2014,10 Sep.,−0.20,2014-09-10
7,2014,11 Jun.,−0.10,2014-06-11


In [13]:
# drop year and day_month columns
df_eur.drop(columns = ['year', 'day_month'], inplace = True)

In [14]:
df_eur.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 3 to 51
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   eur_rate  49 non-null     object        
 1   date      49 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 1.1+ KB


In [15]:
lst_num = '1 2 3 4 5 6 7 8 9 0'.split()


new_r = []
for r in df_eur['eur_rate']:
    if r[0] not in lst_num:
        r = float(r[1:])*(-1)
    else:
        r = float(r)
    
    new_r.append(r)

In [16]:
df_eur['eur_rate'] = new_r

In [17]:
df_eur.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 3 to 51
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   eur_rate  49 non-null     float64       
 1   date      49 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.1 KB


In [18]:
df_eur['eur_rate_lag1'] = df_eur['eur_rate'].shift(1)

In [19]:
df_eur['eur_rate_change'] = df_eur['eur_rate_lag1'] - df_eur['eur_rate']

In [20]:
df_eur

,eur_rate,date,eur_rate_lag1,eur_rate_change
3,-0.50,2019-09-18,NaN,NaN
4,-0.40,2016-03-16,-0.50,-0.10
5,-0.30,2015-12-09,-0.40,-0.10
6,-0.20,2014-09-10,-0.30,-0.10
7,-0.10,2014-06-11,-0.20,-0.10
8,0.00,2013-11-13,-0.10,-0.10
9,0.00,2013-05-08,0.00,0.00
10,0.00,2012-07-11,0.00,0.00
11,0.25,2011-12-14,0.00,-0.25
12,0.50,2011-11-09,0.25,-0.25


In [21]:
%store df_eur

Stored 'df_eur' (DataFrame)
